In [1]:
!nvidia-smi

Mon Jun 10 17:32:07 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.99                 Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   50C    P5             16W /   25W |     823MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [3]:
%pwd

'e:\\Projects for portfolio\\Exoplanet Chatbot\\research'

In [4]:
os.chdir("../") 

In [5]:
%pwd

'e:\\Projects for portfolio\\Exoplanet Chatbot'

In [6]:
# Raw data file
df = pd.read_csv('artifacts\\Exoplanet Data.csv')
df.head()

,Planet Name,Host Name,Planet letter,HD ID,HIP ID,TIC ID,GAIA ID,Number of Stars,Number of Planets,Number of Moons,...,sy_tmag,sy_tmag_reflink,sy_kepmag,sy_kepmag_reflink,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec
0,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,2,1,0,...,3.83790,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,2.0,1,2,0,0,0
1,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,0,...,3.82294,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0.0,1,1,0,0,0
2,14 And b,14 And,b,HD 221345,HIP 116076,TIC 333225860,Gaia DR2 1920113512486282240,1,1,0,...,4.32140,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0.0,1,1,0,0,0
3,14 Her b,14 Her,b,HD 145675,HIP 79248,TIC 219483057,Gaia DR2 1385293808145621504,1,2,0,...,5.86310,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,0.0,1,4,1,0,0
4,16 Cyg B b,16 Cyg B,b,HD 186427,HIP 96901,TIC 27533327,Gaia DR2 2135550755683407232,3,1,0,...,5.62810,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,6.095,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,5.0,1,4,3,0,0


## Feature Selection

In [7]:
columns_to_include = ['Planet Name','Host Name','Number of Stars','Number of Planets','Number of Moons','Circumbinary Flag','Discovery Method', 'Discovery Year',
       'Discovery Publication Date','Discovery Facilty','Discovery Telescope','Planet Radius [Earth Radius]','Planet Mass [Earth Mass]','Planet Density [g/cm**3]','Equilibrium Temperature [K]','Orbit Semi-Major Axis [au]','Radial Velocity Amplitude [m/s]','Stellar Effective Temperature [K]','Stellar Radius [Solar Radius]','Stellar Mass [Solar mass]']

In [8]:
df_final = df[columns_to_include]
df_final.head()

,Planet Name,Host Name,Number of Stars,Number of Planets,Number of Moons,Circumbinary Flag,Discovery Method,Discovery Year,Discovery Publication Date,Discovery Facilty,Discovery Telescope,Planet Radius [Earth Radius],Planet Mass [Earth Mass],Planet Density [g/cm**3],Equilibrium Temperature [K],Orbit Semi-Major Axis [au],Radial Velocity Amplitude [m/s],Stellar Effective Temperature [K],Stellar Radius [Solar Radius],Stellar Mass [Solar mass]
0,11 Com b,11 Com,2,1,0,0,Radial Velocity,2007,2008-01,Xinglong Station,2.16 m Telescope,12.2,4914.89849,14.90,NaN,1.178,288.630,4874.0,13.76,2.09
1,11 UMi b,11 UMi,1,1,0,0,Radial Velocity,2009,2009-10,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,12.3,4684.81420,13.80,NaN,1.530,189.700,4213.0,29.79,2.78
2,14 And b,14 And,1,1,0,0,Radial Velocity,2008,2008-12,Okayama Astrophysical Observatory,1.88 m Telescope,13.1,1131.15130,2.76,NaN,0.775,86.080,4888.0,11.55,1.78
3,14 Her b,14 Her,1,2,0,0,Radial Velocity,2002,2003-01,W. M. Keck Observatory,10 m Keck I Telescope,12.6,2559.47216,7.03,NaN,2.774,89.582,5338.0,0.93,0.91
4,16 Cyg B b,16 Cyg B,3,1,0,0,Radial Velocity,1996,1997-07,Multiple Observatories,Multiple Telescopes,13.5,565.73740,1.26,NaN,1.660,50.500,5750.0,1.13,1.08


## Handling Missing Values

In [9]:
df_final.isnull().sum()

Planet Name                             0
Host Name                               0
Number of Stars                         0
Number of Planets                       0
Number of Moons                         0
Circumbinary Flag                       0
Discovery Method                        0
Discovery Year                          0
Discovery Publication Date              0
Discovery Facilty                       0
Discovery Telescope                     0
Planet Radius [Earth Radius]           21
Planet Mass [Earth Mass]               23
Planet Density [g/cm**3]              112
Equilibrium Temperature [K]          1472
Orbit Semi-Major Axis [au]            288
Radial Velocity Amplitude [m/s]      3518
Stellar Effective Temperature [K]     231
Stellar Radius [Solar Radius]         253
Stellar Mass [Solar mass]               7
dtype: int64

In [10]:
# Filling the null values
df_final.fillna("Not found Yet!!",inplace=True)

C:\Users\arany\AppData\Local\Temp\ipykernel_16348\3067571076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.fillna("Not found Yet!!",inplace=True)


In [11]:
df_final.isnull().sum()

Planet Name                          0
Host Name                            0
Number of Stars                      0
Number of Planets                    0
Number of Moons                      0
Circumbinary Flag                    0
Discovery Method                     0
Discovery Year                       0
Discovery Publication Date           0
Discovery Facilty                    0
Discovery Telescope                  0
Planet Radius [Earth Radius]         0
Planet Mass [Earth Mass]             0
Planet Density [g/cm**3]             0
Equilibrium Temperature [K]          0
Orbit Semi-Major Axis [au]           0
Radial Velocity Amplitude [m/s]      0
Stellar Effective Temperature [K]    0
Stellar Radius [Solar Radius]        0
Stellar Mass [Solar mass]            0
dtype: int64

In [12]:
df_final.head()

,Planet Name,Host Name,Number of Stars,Number of Planets,Number of Moons,Circumbinary Flag,Discovery Method,Discovery Year,Discovery Publication Date,Discovery Facilty,Discovery Telescope,Planet Radius [Earth Radius],Planet Mass [Earth Mass],Planet Density [g/cm**3],Equilibrium Temperature [K],Orbit Semi-Major Axis [au],Radial Velocity Amplitude [m/s],Stellar Effective Temperature [K],Stellar Radius [Solar Radius],Stellar Mass [Solar mass]
0,11 Com b,11 Com,2,1,0,0,Radial Velocity,2007,2008-01,Xinglong Station,2.16 m Telescope,12.2,4914.89849,14.9,Not found Yet!!,1.178,288.63,4874.0,13.76,2.09
1,11 UMi b,11 UMi,1,1,0,0,Radial Velocity,2009,2009-10,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,12.3,4684.8142,13.8,Not found Yet!!,1.53,189.7,4213.0,29.79,2.78
2,14 And b,14 And,1,1,0,0,Radial Velocity,2008,2008-12,Okayama Astrophysical Observatory,1.88 m Telescope,13.1,1131.1513,2.76,Not found Yet!!,0.775,86.08,4888.0,11.55,1.78
3,14 Her b,14 Her,1,2,0,0,Radial Velocity,2002,2003-01,W. M. Keck Observatory,10 m Keck I Telescope,12.6,2559.47216,7.03,Not found Yet!!,2.774,89.582,5338.0,0.93,0.91
4,16 Cyg B b,16 Cyg B,3,1,0,0,Radial Velocity,1996,1997-07,Multiple Observatories,Multiple Telescopes,13.5,565.7374,1.26,Not found Yet!!,1.66,50.5,5750.0,1.13,1.08


## Modifying the Circumbinary Flag column name

In [13]:
df['Circumbinary Flag'].unique() #  Whether the planet orbits a binary system

array([0, 1], dtype=int64)

In [14]:
df_final.rename(columns={'Circumbinary Flag' : 'Binary System'},inplace=True)

C:\Users\arany\AppData\Local\Temp\ipykernel_16348\3598662132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(columns={'Circumbinary Flag' : 'Binary System'},inplace=True)


In [15]:
df_final['Binary System'] = df_final['Binary System'].map({0 : 'Binary System', 1 : 'Not Binary System'})

C:\Users\arany\AppData\Local\Temp\ipykernel_16348\3347119243.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Binary System'] = df_final['Binary System'].map({0 : 'Binary System', 1 : 'Not Binary System'})


## Context Generation

In [16]:
# Function to create a structured context column using the column data
def create_context(row):
    context = (
        f"The exoplanet {row['Planet Name']} orbits the host star {row['Host Name']}. "
        f"The system containing the exoplanet {row['Planet Name']} also contains {row['Number of Stars']} stars, {row['Number of Planets']} planets, and {row['Number of Moons']} moons. "
        f"The exoplanet lies in a {row['Binary System']}. "
        f"The planet was discovered using the {row['Discovery Method']} method in {row['Discovery Year']}. "
        f"The discovery was published on {row['Discovery Publication Date']} and facilitated by {row['Discovery Facilty']} using the {row['Discovery Telescope']}. "
        f"The planet has a radius of {row['Planet Radius [Earth Radius]']} Earth radii, a mass of {row['Planet Mass [Earth Mass]']} Earth masses, "
        f"and a density of {row['Planet Density [g/cm**3]']} g/cm³. "
        f"The equilibrium temperature is {row['Equilibrium Temperature [K]']} K. "
        f"The semi-major axis of its orbit is {row['Orbit Semi-Major Axis [au]']} AU. "
        f"The radial velocity amplitude is {row['Radial Velocity Amplitude [m/s]']} m/s. "
        f"The host star has an effective temperature of {row['Stellar Effective Temperature [K]']} K, "
        f"a radius of {row['Stellar Radius [Solar Radius]']} solar radii, and a mass of {row['Stellar Mass [Solar mass]']} solar masses."
    )
    return context

In [17]:
df_final['Context'] = df_final.apply(create_context, axis=1)

C:\Users\arany\AppData\Local\Temp\ipykernel_16348\4096153124.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Context'] = df_final.apply(create_context, axis=1)


In [18]:
df_final['Context'].head(10)

0    The exoplanet 11 Com b orbits the host star 11...
1    The exoplanet 11 UMi b orbits the host star 11...
2    The exoplanet 14 And b orbits the host star 14...
3    The exoplanet 14 Her b orbits the host star 14...
4    The exoplanet 16 Cyg B b orbits the host star ...
5    The exoplanet 17 Sco b orbits the host star 17...
6    The exoplanet 18 Del b orbits the host star 18...
7    The exoplanet 1RXS J160929.1-210524 b orbits t...
8    The exoplanet 24 Boo b orbits the host star 24...
9    The exoplanet 24 Sex b orbits the host star 24...
Name: Context, dtype: object

In [19]:
df_final.shape

(5638, 21)

In [20]:
len(df_final.columns)

21

In [21]:
## Instruction , Input and Output Q-A pair generated from the data

In [22]:
# Generate instruction, context, and response pairs
instruction_context_response_pairs = []
for index, row in df_final.iterrows():
    planet_name = row['Planet Name']
    context = row['Context']
    features = {
        'Number of Stars': row['Number of Stars'],
        'Number of Planets': row['Number of Planets'],
        'Number of Moons': row['Number of Moons'],
        'Binary System': row['Binary System'],
        'Discovery Method': row['Discovery Method'],
        'Discovery Year': row['Discovery Year'],
        'Discovery Publication Date': row['Discovery Publication Date'],
        'Discovery Facility': row['Discovery Facilty'],
        'Discovery Telescope': row['Discovery Telescope'],
        'Planet Radius': row['Planet Radius [Earth Radius]'],
        'Planet Mass': row['Planet Mass [Earth Mass]'],
        'Planet Density': row['Planet Density [g/cm**3]'],
        'Equilibrium Temperature': row['Equilibrium Temperature [K]'],
        'Orbit Semi-Major Axis': row['Orbit Semi-Major Axis [au]'],
        'Radial Velocity Amplitude': row['Radial Velocity Amplitude [m/s]'],
        'Stellar Effective Temperature': row['Stellar Effective Temperature [K]'],
        'Stellar Radius': row['Stellar Radius [Solar Radius]'],
        'Stellar Mass': row['Stellar Mass [Solar mass]']
    }

    for feature, value in features.items():
        instruction = f"What is the {feature.lower().replace('_', ' ')} of {planet_name}?"
        response = f"The {feature.lower().replace('_', ' ')} of {planet_name} is {value}."
        instruction_context_response_pairs.append({
            "instruction": instruction,
            "input": context,
            "output": response
        })

In [23]:
instruction_dataset = pd.DataFrame(instruction_context_response_pairs)

In [24]:
instruction_dataset.head(20)

,instruction,input,output
0,What is the number of stars of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of stars of 11 Com b is 2.
1,What is the number of planets of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of planets of 11 Com b is 1.
2,What is the number of moons of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of moons of 11 Com b is 0.
3,What is the binary system of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The binary system of 11 Com b is Binary System.
4,What is the discovery method of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The discovery method of 11 Com b is Radial Vel...
5,What is the discovery year of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The discovery year of 11 Com b is 2007.
6,What is the discovery publication date of 11 C...,The exoplanet 11 Com b orbits the host star 11...,The discovery publication date of 11 Com b is ...
7,What is the discovery facility of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The discovery facility of 11 Com b is Xinglong...
8,What is the discovery telescope of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The discovery telescope of 11 Com b is 2.16 m ...
9,What is the planet radius of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The planet radius of 11 Com b is 12.2.


## Saving the Instruction Dataset

In [25]:
instruction_dataset.to_csv('artifacts\\Instruction Q-A pair Exoplanet Data.csv',index=False)